In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
# from scikeras.wrappers import KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LeakyReLU

from keras.models import Sequential
from keras import layers
from keras.callbacks import EarlyStopping
from keras import backend as K
from sklearn.metrics import mean_squared_error
import numpy as np

import sys
import numpy as np
from sklearn.metrics import mean_squared_error
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from keras.models import Sequential
from keras.optimizers import Adadelta, Adam, RMSprop

# Set display options
pd.set_option("display.max_columns", None)

from sklearn.model_selection import KFold


In [2]:
df = pd.read_csv("NFWBS_PUF_2016_data.csv")

In [3]:
def filter_columns(df, include_strings):
    """
    Filter DataFrame columns containing any of the strings in include_strings.
    
    Args:
    df (pandas.DataFrame): Input DataFrame.
    include_strings (list): List of strings that need to be included in the column names.
    
    Returns:
    pandas.DataFrame: Filtered DataFrame.
    """
    filtered_columns = []
    for col in df.columns:
        if any(include_string in col.lower() for include_string in include_strings):
            filtered_columns.append(col)
    return df[filtered_columns]

In [4]:
df = filter_columns(df, include_strings=['fwb'])

In [5]:
def rename_columns(df):
   # convert form camel case to snake case
   df.columns = (df.columns
                  .str.replace('(?<=[a-z])(?=[A-Z])', '_', regex=True)
                  .str.lower()
               )
   # convert " " to _
   df.columns = df.columns.str.replace(' ', '_')
   return df

In [6]:
def drop_negative_values(df):
    """
    Drop all values in the DataFrame that are less than 0.

    Parameters:
        df (pandas.DataFrame): Input DataFrame.

    Returns:
        pandas.DataFrame: DataFrame with negative values dropped.
    """
    df_without_null = df[df >= 0]
    print(len(df)-len(df_without_null.dropna()))
    return df_without_null.dropna()

In [7]:
df = drop_negative_values(df)

19


In [8]:
target_column = [col for col in df.columns if 'score' in col.lower()]
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

train = train_set.drop(columns=target_column, errors='ignore')
train_labels = train_set[target_column]

test = test_set.drop(columns=target_column, errors='ignore')
test_labels = test_set[target_column]

# ANN Hyperparameter tuning

In [37]:
import hyperopt
from hyperopt import hp, fmin, tpe, Trials, space_eval, STATUS_OK

In [59]:
space = {
    'hiddenLayerOne': hp.uniform('hiddenLayerOne', 64, 1024),
    'hiddenLayerTwo': hp.uniform('hiddenLayerTwo', 64, 1024),
    'dropout': hp.uniform('dropout', 0.25, 0.75),
    'batch_size': hp.uniform('batch_size', 28, 128),
    'nb_epochs': 100,
    'optimizer': hp.choice('optimizer', ['adadelta', 'adam', 'rmsprop']),
    'activation': 'relu'
}

def f_nn(params):
    print('Params testing: ', params)
    model = Sequential([
        Dense(int(params['hiddenLayerOne']), activation=params['activation'], input_shape=(train.shape[1],)),  # Input layer
        Dropout(params['dropout']),
        Dense(int(params['hiddenLayerTwo']), activation=params['activation']),  # Hidden layer
        Dropout(params['dropout']),
        Dense(1, activation='linear')  # Output layer for numerical prediction
    ])

    model.compile(loss='mean_squared_error', optimizer=params['optimizer'])

    model.fit(train, train_labels, epochs=params['nb_epochs'], batch_size=int(params['batch_size']), verbose=0)

    y_pred = model.predict(train)
    mse_train = mean_squared_error(train_labels, y_pred)

    return {'loss': mse_train, 'status': STATUS_OK}

trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=50, trials=trials)
print('Best parameters:', best)

Params testing:                                       
{'activation': 'relu', 'batch_size': 77.89439400341456, 'dropout': 0.4487145443722915, 'hiddenLayerOne': 264.7566015978515, 'hiddenLayerTwo': 948.8122591534299, 'nb_epochs': 100, 'optimizer': 'rmsprop'}
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

Cause: Unable to locate the source code of <function Model.make_predict_function.<locals>.predict_function at 0x000001B83D4B47C0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: Unable to locate the source code of <function Model.make_predict_function.<locals>.predict_function at 0x000001B83D4B47C0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Orig

Best parameters: {'batch_size': 94.35859333868757, 'dropout': 0.3139458037685006, 'hiddenLayerOne': 782.0839785246619, 'hiddenLayerTwo': 500.88142737977785, 'optimizer': 1}

In [53]:
print(train.columns)

Index(['FWB1_1', 'FWB1_2', 'FWB1_3', 'FWB1_4', 'FWB1_5', 'FWB1_6', 'FWB2_1',
       'FWB2_2', 'FWB2_3', 'FWB2_4'],
      dtype='object')
